<a href="https://colab.research.google.com/github/ajimper/ProastronomiaAstroWorkshop/blob/main/HR_Diagram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install astroquery astropy matplotlib numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astroquery.vizier import Vizier
from astropy import units as u
from astropy.table import QTable

In [ ]:
class Star:
    def __init__(self, hip, vmag, bv, plx, sptype, name=None):
        self.hip = hip
        self.name = name
        self.vmag = vmag * u.mag
        self.bv = bv * u.dimensionless_unscaled
        self.plx = plx * u.mas
        self.sptype = sptype

        self._calculate_parameters()

    def _calculate_parameters(self):
        # Calcular distancia
        #self.distance = (1/self.plx).to(u.pc) if self.plx > 0 else np.nan * u.pc
        if (self.plx > 0 ):
            self.distance = (self.plx).to(u.parsec, equivalencies=u.parallax())
        else
            else np.nan * u.parsec

        # Calcular magnitud absoluta
        self.Mv = self.vmag + 5 * np.log10(self.distance/10 * u.pc)

        # Calcular temperatura efectiva (Casagrande et al. 2010)
        self.Teff = 9100 * u.K * (self.bv + 0.65)**(-0.66)

        # Calcular luminosidad (usando relación bolométrica aproximada)
        BC = -0.08 * self.Mv.value**2 + 0.70 * self.Mv.value - 1.37  # Corrección bolométrica
        M_bol = self.Mv + BC
        self.L = 10**(-0.4 * (M_bol - 4.74)) * u.L_sun

    @property
    def log_Teff(self):
        return np.log10(self.Teff.value)

    @property
    def log_L_Lsun(self):
        return np.log10(self.L.value)

In [ ]:
# Consulta al catálogo Hipparcos
Vizier.ROW_LIMIT = 200
v = Vizier(columns=['HIP', 'Vmag', 'B-V', 'Plx', 'e_Plx', 'SpType', 'VarType', 'RAJ2000', 'DEJ2000'],
           column_filters={'B-V': '>0', 'Plx': '>10', 'Vmag': '<6'})
catalogs = v.get_catalogs('I/239/hip_main')
star_catalogue = catalogs[0].filled()

In [ ]:
# Filtrar y crear objetos Star
stars = []
known_names = {  # Algunos nombres propios conocidos
    32349: 'Sirio',
    91262: 'Vega',
    97649: 'Altair',
    24436: 'Betelgeuse',
    24608: 'Rigel',
    80763: 'Arcturus'
}

In [ ]:
for row in star_catalogue:
    #if np.isfinite(row['B-V']) and row['Plx']/row['e_Plx'] > 5:
    if np.isfinite(row['B-V']):
        star = Star(
            hip=row['HIP'],
            vmag=row['Vmag'],
            bv=row['B-V'],
            plx=row['Plx'],
            sptype=row['SpType'],
            name=known_names.get(row['HIP'])
        )
        stars.append(star)

UnitConversionError: '1 / mas' and 'pc' (length) are not convertible

In [ ]:
# Crear QTable con resultados
table = QTable()
table['HIP'] = [s.hip for s in stars]
table['Nombre'] = [s.name if s.name else f'HIP {s.hip}' for s in stars]
table['log_Teff'] = [s.log_Teff for s in stars] * u.dimensionless_unscaled
table['log_L_Lsun'] = [s.log_L_Lsun for s in stars] * u.dimensionless_unscaled
table['B-V'] = [s.bv.value for s in stars] * u.dimensionless_unscaled

In [ ]:
# Configurar el gráfico
plt.figure(figsize=(14, 10), facecolor='black')
ax = plt.gca()
ax.set_facecolor('black')

In [ ]:
# Crear fondo con gradación de colores
x = np.linspace(3.4, 4.0, 100)
y = np.linspace(-6, 6, 100)
X, Y = np.meshgrid(x, y)
Z = X  # Usamos la temperatura para el color
plt.pcolormesh(X, Y, Z, cmap='plasma', alpha=0.2, shading='auto')

In [ ]:
# Graficar estrellas
sc = plt.scatter(
    table['log_Teff'],
    table['log_L_Lsun'],
    c=table['B-V'],
    cmap='viridis',
    s=150,
    edgecolor='white',
    alpha=0.9
)

In [ ]:
# Añadir etiquetas de nombres
for star in stars:
    if star.name:
        plt.text(
            star.log_Teff + 0.005,
            star.log_L_Lsun,
            star.name,
            color='white',
            fontsize=9,
            ha='left',
            va='center'
        )

In [ ]:
# Configuraciones estéticas
plt.colorbar(sc, label='Índice de color B-V', pad=0.02)
plt.gca().invert_xaxis()
plt.title('Diagrama Hertzsprung-Russell\nlog(T$_{eff}$) vs log(L/L$_\odot$)',
         color='white', fontsize=14, pad=20)
plt.xlabel('log(T$_{eff}$) [K]', color='white')
plt.ylabel('log(L/L$_\odot$)', color='white')
plt.xticks(color='white')
plt.yticks(color='white')

In [ ]:
# Añadir líneas de referencia
plt.axhline(0, color='gray', linestyle='--', alpha=0.5)
plt.axvline(3.7, color='gray', linestyle='--', alpha=0.5)

In [ ]:
# Mostrar gráfico
plt.tight_layout()
plt.show()